In [1]:
def generate_char_list(start_char, end_char, step):
    """
    Generates a list of characters from start_char to end_char with a given step.
    
    Parameters:
    start_char (str): The starting character.
    end_char (str): The ending character.
    step (int): The step between characters.
    
    Returns:
    list: List of characters.
    """
    start = ord(start_char.upper())
    end = ord(end_char.upper())
    return [chr(i) for i in range(start, end + 1, step)]

# Contoh penggunaan
start_char = 'D'
end_char = 'M'
step = 2

# result = generate_char_list(start_char, end_char, step)
# print(result)  # Output: ['D', 'F', 'H']

col_absen = generate_char_list('D','M',2)
for char in generate_char_list('O','X',2):
    col_absen.append(char)

print(col_absen)

['D', 'F', 'H', 'J', 'L', 'O', 'Q', 'S', 'U', 'W']


In [20]:
import pandas as pd
from openpyxl import load_workbook

def edit_excel(file_path, dataframe, kolom_ujian, target_path):
    # Load workbook and select the active worksheet
    workbook = load_workbook(file_path)
    sheet = workbook.active
    
    # Define the starting row and ending row
    start_row = 18
    end_row = 45
    
    # Iterate over the rows and update the values from the dataframe
    for idx, row in enumerate(range(start_row, end_row + 1)):
        if idx < len(dataframe):
            sheet[f'B{row}'] = dataframe.iloc[idx]['NPM']
            sheet[f'C{row}'] = dataframe.iloc[idx]['Nama Lengkap']
            sheet[f'D{row}'] = dataframe.iloc[idx][kolom_ujian]
    
    # Save the workbook
    workbook.save(target_path)

def edit_excel_2(file_path, dataframe, target_path, uts_col, uas_col, praktikum_col):
    # Load workbook and select the active worksheet
    workbook = load_workbook(file_path)
    sheet = workbook.active
    
    # Define the starting row and ending row
    start_row = 13
    end_row = 40
    
    # Iterate over the rows and update the values from the dataframe
    for idx, row in enumerate(range(start_row, end_row + 1)):
        if idx < len(dataframe):
            col_absen = generate_char_list('D','M',2)
            for char in generate_char_list('O','X',2):
                col_absen.append(char)

            col_tugas = generate_char_list('E','M',2)
            for char in generate_char_list('P','X',2):
                col_tugas.append(char)

            for char in col_absen:
                sheet[f'{char}{row}'] = dataframe.iloc[idx]["NILAI ABSEN"]

            for ii, char in enumerate(col_tugas):
                sheet[f'{char}{row}'] = dataframe.iloc[idx][praktikum_col[ii]]

            sheet[f'B{row}'] = dataframe.iloc[idx]['NPM']
            sheet[f'C{row}'] = dataframe.iloc[idx]['Nama Lengkap']
            sheet[f'N{row}'] = dataframe.iloc[idx][uts_col]
            sheet[f'Y{row}'] = dataframe.iloc[idx][uas_col]
            sheet[f'AA{row}'] = dataframe.iloc[idx][uts_col]
            sheet[f'AB{row}'] = dataframe.iloc[idx][uas_col]
            sheet[f'AE{row}'] = dataframe.iloc[idx][uts_col]
            sheet[f'AF{row}'] = dataframe.iloc[idx][uas_col]
            sheet[f'AC{row}'] = dataframe.iloc[idx]["NILAI ABSEN"] * 10
            sheet[f'AG{row}'] = dataframe.iloc[idx]["NILAI ABSEN"]
            sheet[f'AH{row}'] = dataframe.iloc[idx]["TOTAL NILAI(%)"]
            sheet[f'AD{row}'] = dataframe.iloc[idx]["TOTAL NILAI PRAKTIKUM(%)"]
            sheet[f'Z{row}'] = dataframe.iloc[idx]["TOTAL NILAI PRAKTIKUM"]
    
    # Save the workbook
    workbook.save(target_path)

"""
TUGAS PRAKTIKUM = 30%
UTS PRAKTIKUM = 30%
UAS PRAKTIKUM = 40%
"""

def rekapitulasi_nilai(path_file_nilai, path_file_uas, target_path_file):

    uts_col = "UTS Praktikum"
    uas_col = "UAS Praktikum"

    def rekap_nilai(tugas, uts, uas):
        absen = 100.0
        return (float(tugas) * 0.2) + (float(uts) * 0.3) + (float(uas) * 0.4) + (float(absen) * 0.1)

    def rekap_nilai_praktikum(list_nilai):
        return sum(list_nilai) / len(list_nilai)

    df = pd.read_csv(path_file_nilai, delimiter=",", on_bad_lines='skip')
    df_uas = pd.read_excel(path_file_uas)
    try:
        df_uas["NPM"] = [(df_uas["Alamat Email"][i]).split("@")[0] for i in range(len(df))]
    except:
        pass
    df = df.iloc[2:]
    df = df.reset_index(drop=True)

    nama_col = ['Nama Belakang', 'Nama Depan']
    for col in nama_col:
        df[col] = df[col].fillna("")

    df["Nama Lengkap"] = ""

    for i in range(len(df)):
        try:
            df["Nama Lengkap"][i] = str(str(df["Nama Depan"][i]) + " " + str(df['Nama Belakang'][i])).split("_")[1].upper()
        except:
            df["Nama Lengkap"][i] = str(str(df["Nama Depan"][i]) + " " + str(df['Nama Belakang'][i])).upper()
    
    all_col = ['Nama Belakang', 'Nama Depan', "Nama Lengkap", 'Alamat Email']
    for col in df.columns:
        if "Praktikum" in col or "Practicum" in col:
            all_col.append(col)

    df = df[all_col]

    df["NPM"] = [(df["Alamat Email"][i]).split("@")[0] for i in range(len(df))]
    df_uas["NPM"] = [str(df_uas["NPM"][i]) for i in range(len(df_uas))]
    df_merge = pd.merge(df, df_uas, on="NPM", how="inner")

    df[uas_col] = df_merge["Total"]
    df['NILAI ABSEN'] = 100.0
    df = df.fillna(0.0)

    praktikum_col = [col for col in df.columns if ("UTS" not in col) and ("UAS" not in col) and ("Praktikum" in col) or ("Practicum" in col)]
    df['TOTAL NILAI PRAKTIKUM'] = [sum([float(df[col][i]) for col in praktikum_col]) for i in range(len(df))]
    df['TOTAL NILAI PRAKTIKUM(%)'] = [rekap_nilai_praktikum([float(df[col][i]) for col in praktikum_col]) for i in range(len(df))]
    df['TOTAL NILAI(%)'] = [rekap_nilai(df['TOTAL NILAI PRAKTIKUM(%)'][i], df[uts_col][i], df[uas_col][i]) for i in range(len(df))]

    dir_target = path_file_nilai.split("/")[0]
    target_path = f"{dir_target}/Daftar Nilai UTS {dir_target}.xlsx"
    edit_excel("Daftar Nilai UTS.xlsx", df, uts_col, target_path)
    target_path = f"{dir_target}/Daftar Nilai UAS {dir_target}.xlsx"
    edit_excel("Daftar Nilai UAS.xlsx", df, uas_col, target_path)
    target_path = f"{dir_target}/Rekapitulasi Nilai Praktikum {dir_target}.xlsx"
    edit_excel_2("Rekapitulasi Nilai Praktikum.xlsx", df, target_path, uts_col, uas_col, praktikum_col)

    return df, df_merge
    # df.to_excel(target_path_file, index=False)

In [ ]:
df, df2 = rekapitulasi_nilai("4B/Informatika_4B_Kecerdasan_Buatan.csv", "4B/Nilai UAS Praktikum Kecerdasan Buatan 4B.xlsx", "")

In [21]:
df3, df4 = rekapitulasi_nilai("4C/Informatika_4C_Kecerdasan_Buatan.csv", "4C/Nilai UAS Praktikum Kecerdasan Buatan 4C.xlsx", "")

/tmp/ipykernel_5925/1387121350.py:101: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df["Nama Lengkap"][i] = str(str(df["Nama Depan"][i]) + " " + str(df['Nama Belakang'][i])).split("_")[1].upper()
/tmp/ipykernel_5925/1387121350.py:101: Futur

In [19]:
df3

,Nama Belakang,Nama Depan,Nama Lengkap,Alamat Email,Praktikum 1,Praktikum 2,Praktikum 3,Praktikum 4,Assignment Practicum 5 - AI Study Case,UTS Praktikum,...,Praktikum 8,Praktikum 9,Praktikum 10,NPM,Kelompok,Nama,Presentasi(15),Tanya Jawab(5),Kode(80),Total
0,Maulana,3C_Emil,EMIL MAULANA,2210631170019@student.unsika.ac.id,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2210631170019,NOOB TEAM,Daniel Agus Satrio,75,0,93,85.65
1,RIZQI FAJRI,3C_MUHAMAD,MUHAMAD RIZQI FAJRI,2210631170081@student.unsika.ac.id,100.0,85.0,100.0,100.0,95.0,100.0,...,100.0,100.0,100.0,2210631170081,Overlord,Muhamad Rizqi Fajri,95,100,90,91.25
2,ADAM SANDY AZHARI,4C_,ADAM SANDY AZHARI,2210631170108@student.unsika.ac.id,NaN,85.0,NaN,90.0,95.0,100.0,...,100.0,100.0,NaN,2210631170108,Overlord,Adam Shandy Azhari,75,0,90,83.25
3,RIZKY DARMAWAN,4C_ADITYA,ADITYA RIZKY DARMAWAN,2210631170109@student.unsika.ac.id,100.0,95.0,100.0,100.0,95.0,100.0,...,100.0,100.0,100.0,2210631170109,SatPAM,Aditya Rizky Darmawan,95,100,95,95.25
4,TUBAGUS NABIL MAULANA,4C_AHMAD,AHMAD TUBAGUS NABIL MAULANA,2210631170110@student.unsika.ac.id,90.0,60.0,100.0,85.0,95.0,100.0,...,100.0,100.0,100.0,2210631170110,OneWeek,Ahmad Tubagus Nabil Maulana,95,0,100,94.25
5,PUTRI NADILA GUSTIN,4C_ALLYA,ALLYA PUTRI NADILA GUSTIN,2210631170111@student.unsika.ac.id,100.0,90.0,100.0,100.0,100.0,100.0,...,100.0,100.0,100.0,2210631170111,HALARA,Allya Putri Nadilla Gustin,85,20,95,89.75
6,KHOERIAH UTAMI,4C_BUNGA,BUNGA KHOERIAH UTAMI,2210631170113@student.unsika.ac.id,100.0,90.0,100.0,50.0,95.0,100.0,...,100.0,100.0,100.0,2210631170113,NOOB TEAM,Bunga Khoeriah Utami,95,100,92,92.85
7,Ahmad Pudoli,4C_Chandra,CHANDRA AHMAD PUDOLI,2210631170114@student.unsika.ac.id,100.0,90.0,100.0,85.0,95.0,100.0,...,100.0,100.0,100.0,2210631170114,NOOB TEAM,Chandra Ahmad Pudoli,95,0,90,86.25
8,AGUS SATRIO,4C_DANIEL,DANIEL AGUS SATRIO,2210631170017@student.unsika.ac.id,90.0,95.0,100.0,NaN,NaN,80.0,...,NaN,NaN,NaN,2210631170017,NaN,Emil Maulana,0,0,0,0.00
9,,4C_Daffa Erinda Ziddana,DAFFA ERINDA ZIDDANA,2210631170115@student.unsika.ac.id,100.0,90.0,100.0,85.0,95.0,100.0,...,100.0,100.0,100.0,2210631170115,SatPAM,Daffa Erinda Zidana,75,100,95,92.25
